# Worksheet: time series data exploration
Data and examples from https://keras.io/examples/timeseries/timeseries_weather_forecasting/ 

In [ ]:
#! pip install seaborn

In [ ]:
# import necessary modules 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras

In [ ]:
# download data 
from zipfile import ZipFile

uri = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip"
zip_path = keras.utils.get_file(origin=uri, fname="jena_climate_2009_2016.csv.zip")
zip_file = ZipFile(zip_path)
zip_file.extractall()
csv_path = "jena_climate_2009_2016.csv"

df = pd.read_csv(csv_path)

The table below shows the column names, their value formats, and their description.

| Index |     Features    |        Format       |                                                                                                      Description                                                                                                     |
|:-----:|:---------------:|:-------------------:|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| 1     | Date Time       | 01.01.2009 00:10:00 | Date-time reference                                                                                                                                                                                                  |
| 2     | p (mbar)        | 996.52              | The pascal SI derived unit of pressure used to quantify internal  pressure. Meteorological reports typically state atmospheric pressure in  millibars.                                                               |
| 3     | T (degC)        | -8.02               | Temperature in Celsius                                                                                                                                                                                               |
| 4     | Tpot (K)        | 265.4               | Temperature in Kelvin                                                                                                                                                                                                |
| 5     | Tdew (degC)     | -8.9                | Temperature in Celsius relative to humidity. Dew Point is a measure  of the absolute amount of water in the air, the DP is the temperature at  which the air cannot hold all the moisture in it and water condenses. |
| 6     | rh (%)          | 93.3                | Relative Humidity is a measure of how saturated the air is with  water vapor, the %RH determines the amount of water contained within  collection objects.                                                           |
| 7     | VPmax (mbar)    | 3.33                | Saturation vapor pressure                                                                                                                                                                                            |
| 8     | VPact (mbar)    | 3.11                | Vapor pressure                                                                                                                                                                                                       |
| 9     | VPdef (mbar)    | 0.22                | Vapor pressure deficit                                                                                                                                                                                               |
| 10    | sh (g/kg)       | 1.94                | Specific humidity                                                                                                                                                                                                    |
| 11    | H2OC (mmol/mol) | 3.12                | Water vapor concentration                                                                                                                                                                                            |
| 12    | rho (g/m ** 3)  | 1307.75             | Airtight                                                                                                                                                                                                             |
| 13    | wv (m/s)        | 1.03                | Wind speed                                                                                                                                                                                                           |
| 14    | max. wv (m/s)   | 1.75                | Maximum wind speed                                                                                                                                                                                                   |
| 15    | wd (deg)        | 152.3               | Wind direction in degrees                                                                                                                                                                                            |


In [ ]:
# visualise the raw data 
titles = [
    "Pressure",
    "Temperature",
    "Temperature in Kelvin",
    "Temperature (dew point)",
    "Relative Humidity",
    "Saturation vapor pressure",
    "Vapor pressure",
    "Vapor pressure deficit",
    "Specific humidity",
    "Water vapor concentration",
    "Airtight",
    "Wind speed",
    "Maximum wind speed",
    "Wind direction in degrees",
]

feature_keys = [
    "p (mbar)",
    "T (degC)",
    "Tpot (K)",
    "Tdew (degC)",
    "rh (%)",
    "VPmax (mbar)",
    "VPact (mbar)",
    "VPdef (mbar)",
    "sh (g/kg)",
    "H2OC (mmol/mol)",
    "rho (g/m**3)",
    "wv (m/s)",
    "max. wv (m/s)",
    "wd (deg)",
]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "purple",
    "brown",
    "pink",
    "gray",
    "olive",
    "cyan",
]

date_time_key = "Date Time"


def show_raw_visualization(data):
    time_data = data[date_time_key]
    fig, axes = plt.subplots(
        nrows=7, ncols=2, figsize=(15, 20), dpi=80, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)):
        key = feature_keys[i]
        c = colors[i % (len(colors))]
        t_data = data[key]
        t_data.index = time_data
        t_data.head()
        ax = t_data.plot(
            ax=axes[i // 2, i % 2],
            color=c,
            title="{} - {}".format(titles[i], key),
            rot=25,
        )
        ax.legend([titles[i]])
    plt.tight_layout()


show_raw_visualization(df)


In [ ]:
# see how data look like 
df

In [ ]:
# set date time as the index 
df = df.set_index(['Date Time'])
df

In [ ]:
# show summary of the data 
df.describe()

In [ ]:
# clean bad values 
df = df[df['wv (m/s)'] != -9999.0]
df = df[df['max. wv (m/s)'] != -9999.0]
df

In [ ]:
# visualise the data after cleaning
df = df.reset_index() # reset the index to use the visualisation function 
show_raw_visualization(df)

In [ ]:
# see linear correlation between the variables 
df = df.set_index(['Date Time'])
corr = df.corr()

## Visualise correlation matrix (linear/Pearson correlation)

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap='bwr', vmax=1.0, vmin=-1.0, center=0,
                 square=True, linewidths=.5, cbar_kws={"shrink": .5}, 
                 annot=True, fmt=".1f") 
plt.show()

In [ ]:
# take a reduced dataset to work with less variables for now
reduced_df = df[['p (mbar)', 'T (degC)', 'rh (%)', 'H2OC (mmol/mol)', 'wv (m/s)']]
reduced_df

In [ ]:
# Rename the columns 
reduced_df.columns = ['Pressure', 'Temperature','RelativeHumidity', 'WaterConcentration', 'WindSpeed']
reduced_df

In [ ]:
# Plot the correlation. There should be less redundancy this time. 
corr = reduced_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8, 8))


# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap='bwr', vmax=1.0, vmin=-1.0, center=0,
                 square=True, linewidths=.5, cbar_kws={"shrink": .5}, 
                 annot=True, fmt=".1f") 
plt.show()

## Explore Nonlinear Correlation with Mutual Information 

In [ ]:
from sklearn.feature_selection import mutual_info_regression as mi_reg

In [ ]:
# define independent variables in the reduced dataset 
indep_vars = ['Pressure','RelativeHumidity', 'WaterConcentration', 'WindSpeed'] # set independent vars
dep_vars = reduced_df.columns.difference(indep_vars).tolist() # set dependent vars
print(indep_vars)
print(dep_vars)

In [ ]:
# Compute mutual information (nonlinear correlation) between independent and dependent variables 
reduceddf_mi = pd.DataFrame([mi_reg(reduced_df[indep_vars], reduced_df[dep_var]) for dep_var in dep_vars], index = dep_vars, columns = indep_vars).apply(lambda x: x / x.max(), axis = 1)
reduceddf_mi

### Plot the mutual information 

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(8,3))

ax = sns.heatmap(reduceddf_mi, square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True, fmt=".1f", cmap='turbo', vmin=0.0, vmax=1.0)
plt.show()

# Try modeling the temperature with random forest regression 
### Since the dataset is large, we will resample it first to work with a smaller dataset. 

### Format the "Date Time" so that it is understood by Pandas. Then we make it as the index for easy handling. 

In [ ]:
reduced_df = reduced_df.reset_index()
reduced_df['Date Time'] = pd.to_datetime(reduced_df['Date Time'], dayfirst=True)
reduced_df = reduced_df.set_index(['Date Time'])
reduced_df

### Resample from 10-min cadence to 1-hour cadence using hourly mean

In [ ]:
# Resample data using hourly mean 
reduced_df = reduced_df.resample('1h').mean().interpolate()
reduced_df

### Visualise the data

In [ ]:
input_features = ['Pressure','RelativeHumidity', 'WaterConcentration','WindSpeed']
output_target = ['Temperature']

In [ ]:
fig, ax = plt.subplots(len(input_features), 2, figsize=(12, 8))
plt.subplots_adjust(hspace=0.5)

i = 0
for col in input_features:
    if i < len(input_features):
        ax[i,0].plot(reduced_df[col], label=col)
        ax[i,0].legend(loc='upper right',bbox_to_anchor=(1.0, 1.0))
        i += 1 
    
j = 0
for col in output_target:
    if j < len(output_target):
        ax[j,1].plot(reduced_df[col], label=col)
        ax[j,1].legend(loc='upper right',bbox_to_anchor=(1.0, 1.0))
        j += 1
    

ax[0,0].set_title('Input features')

ax[0,1].set_title('Output targets')

plt.show()

## Divide the data to train:validation:test

In [ ]:
from datetime import datetime

In [ ]:
# fix column order 
reduced_df = reduced_df[['Pressure','RelativeHumidity', 'WaterConcentration','WindSpeed','Temperature']]

# define training data from 2009 to end of 2012 (5 years)
reduced_df_train = reduced_df[datetime(2009,1,1,0,0,0):datetime(2012,12,31,23,0,0)]

# define validation data in 2013 
reduced_df_val = reduced_df[datetime(2013,1,1,0,0,0):datetime(2013,12,31,23,0,0)]

# define test data in 2014 
reduced_df_test = reduced_df[datetime(2014,1,1,0,0,0):datetime(2014,12,31,23,0,0)]

## Obtain scaling from the training data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## Scale the data frame
scaler = MinMaxScaler()

## Obtain scaler based on the "train" data
df_train_scaled = scaler.fit_transform(reduced_df_train)

## Convert from dataframe to 2D array
print('Train data: \n',df_train_scaled)
print(df_train_scaled.shape)

### Save the scaler for use later

In [ ]:
from pickle import dump

# save the scaler
dump(scaler, open('Scaler_temperature-prediction.pkl', 'wb'))

### Apply the scaling obtained from the "train" data to "validation" and "test" data

In [ ]:
df_val_scaled = scaler.transform(reduced_df_val)
df_test_scaled = scaler.transform(reduced_df_test)

print('Validation data: \n',df_val_scaled)
print(df_val_scaled.shape)

print('Test data: \n',df_test_scaled)
print(df_test_scaled.shape)

## Define X (independent variables) and y (dependent variables)

### Training data 

In [ ]:
## Get input 'X'
X_train = df_train_scaled[:,0:len(input_features)]

print('X train = \n',X_train)
print(X_train.shape)

## Get output 'y'
y_train = df_train_scaled[:,len(input_features):len(input_features)+len(output_target)]

print('y train = \n',y_train)
print(y_train.shape)

### Validation and test data 

In [ ]:
X_val = df_val_scaled[:,0:len(input_features)]
X_test = df_test_scaled[:,0:len(input_features)]

y_val = df_val_scaled[:,len(input_features):len(input_features)+len(output_target)]
y_test = df_test_scaled[:,len(input_features):len(input_features)+len(output_target)]

# Train the model with Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
%%time

model = RandomForestRegressor()

# Fit the model with Random Forest Regressor 
model.fit(X_train, y_train.ravel())

# Evaluate model performance on the train and validation data 
r2_train = model.score(X_train, y_train.ravel())
r2_val = model.score(X_val, y_val.ravel())

print(f"model score on training data: {r2_train}")
print(f"model score on validation data: {r2_val}")

## With Random Forest, it is possible to obtain feature importance 

In [ ]:
importances = model.feature_importances_

indices = np.argsort(importances)

fig, ax = plt.subplots()
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
_ = ax.set_yticklabels(np.array(input_features)[indices])
ax.set_title('Relative Importance for {} (training set)'.format(output_target))

plt.show()


### Model prediction 

In [ ]:
y_predict = model.predict(X_test)
y_predict

## Quickly visualise the result on the test set

In [ ]:
plt.plot(y_test, label='Real data')
plt.plot(y_predict, label='Prediction')
plt.legend()
plt.show()

## Scale back the data 

## Concatenate the scaled 'X_test' and the 'y_predict' for using the inverse scaling

In [ ]:
predict_scaled = np.concatenate((X_test,y_predict.reshape((len(y_predict), 1))), axis=1)
print(predict_scaled.shape)
print(predict_scaled)

## Perform inverse scaling

In [ ]:
predict = scaler.inverse_transform(predict_scaled)
print(predict)
print(predict.shape)

## Make a dataframe for the output

In [ ]:
df_predict = pd.DataFrame(predict)
df_predict.columns = ['Pressure','RelativeHumidity', 'WaterConcentration','WindSpeed', 'Temperature']
df_predict['time'] = reduced_df_test.index
df_predict = df_predict.set_index(['time'])
df_predict

## Visualise model prediction 

In [ ]:
fig = plt.figure(figsize=(10,4))
plt.subplots_adjust(hspace=0.2)

ax1 = fig.add_subplot(1,1,1)
ax1.plot(reduced_df_test['Temperature'], label='True')
ax1.plot(df_predict['Temperature'], label='Prediction')
ax1.legend()
ax1.set_xlabel('Time')
ax1.set_ylabel('Degree C')

plt.show()